In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dataset = pd.read_csv("train.csv")
dataset.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
dataset.location.value_counts()

USA                 104
New York             71
United States        50
London               45
Canada               29
                   ... 
N?? Y???.             1
Wasington, DC         1
Texas af              1
Fort Collins, CO      1
RhodeIsland           1
Name: location, Length: 3341, dtype: int64

In [ ]:
dataset.dtypes

id           int64
keyword     object
location    object
text        object
target       int64
dtype: object

In [ ]:
dataset.dropna(subset=['keyword'], inplace=True)

In [ ]:
dataset.reset_index(inplace=True)

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7552 entries, 0 to 7551
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     7552 non-null   int64 
 1   id        7552 non-null   int64 
 2   keyword   7552 non-null   object
 3   location  5080 non-null   object
 4   text      7552 non-null   object
 5   target    7552 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 354.1+ KB


In [ ]:
keywords = dataset['keyword']

In [ ]:
keywords = np.array(keywords)
keywords = keywords.reshape(-1, 1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
enc.fit(keywords)

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [ ]:
enc.categories_

[array(['ablaze', 'accident', 'aftershock', 'airplane%20accident',
        'ambulance', 'annihilated', 'annihilation', 'apocalypse',
        'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked',
        'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze',
        'blazing', 'bleeding', 'blew%20up', 'blight', 'blizzard', 'blood',
        'bloody', 'blown%20up', 'body%20bag', 'body%20bagging',
        'body%20bags', 'bomb', 'bombed', 'bombing', 'bridge%20collapse',
        'buildings%20burning', 'buildings%20on%20fire', 'burned',
        'burning', 'burning%20buildings', 'bush%20fires', 'casualties',
        'casualty', 'catastrophe', 'catastrophic', 'chemical%20emergency',
        'cliff%20fall', 'collapse', 'collapsed', 'collide', 'collided',
        'collision', 'crash', 'crashed', 'crush', 'crushed', 'curfew',
        'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths', 'debris',
        'deluge', 'deluged', 'demolish', 'demolished', 'demolition',
        'der

In [ ]:
keywords = (enc.transform(keywords)).toarray()

In [ ]:
keywords

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
dataset.head()

,index,id,keyword,location,text,target
0,31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
1,32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
2,33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
3,34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
4,35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0


In [ ]:
import re
import nltk
nltk.download("stopwords") # downloads stopwords
from nltk.corpus import stopwords # imports them afterwards
from nltk.stem.porter import PorterStemmer # applies stemming, takes only roots of the words
corpus = [] # it will contain all our different tweets, but all clean
ps = PorterStemmer()
all_stopwords = stopwords.words('english')[:143]
all_stopwords.remove('not')
all_stopwords.remove("don")
all_stopwords.remove("don't")
for i in range(0, len(dataset)):
  tweet = re.sub('[^a-zA-Z]', ' ', str(dataset['text'][i]) + str(dataset['location'][i]))
  tweet = tweet.lower()
  tweet = tweet.split()
  tweet = [ps.stem(word) for word in tweet if not word in set(all_stopwords)]
  tweet = ' '.join(tweet)
  corpus.append(tweet)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)

X = cv.fit_transform(corpus).toarray()
y = dataset["target"].values

In [ ]:
X = np.concatenate((keywords, X), axis=1)

In [ ]:
X[:20]

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
f1_scores = {}

# Logistic Regression

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import RandomizedSearchCV

# lr_grid = {"C": np.logspace(-4, 4, 40),
#                 "penalty": ['l1', 'l2'],
#                 "solver": ["liblinear"]}

# lr_search = RandomizedSearchCV(estimator=LogisticRegression(),
#                           param_distributions=lr_grid,
#                           cv=5,
#                           scoring='f1',
#                           verbose=True)

# lr_search.fit(X_train, y_train)
# lr_best_params = lr_search.best_params_

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(penalty = 'l2',solver = 'liblinear', max_iter=1000)
lr_model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred_lr = lr_model.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score

f1_score_lr = f1_score(y_test, y_pred_lr)
print(f1_score_lr)

0.7360655737704918


In [ ]:
f1_scores["LogisticRegression"] = f1_score_lr

# K-neighbors

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier

# knc_grid = {"leaf_size": list(range(1, 5)),
#             "n_neighbors": list(range(1, 5)),
#             "p": [2]}

# knc_search = RandomizedSearchCV(estimator=KNeighborsClassifier(),
#                           param_distributions=knc_grid,
#                           cv=5,
#                           scoring='f1',
#                           verbose=True)

# knc_search.fit(X_train, y_train)
# knc_best_params = knc_search.best_params_

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knc_model = KNeighborsClassifier()
knc_model.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
y_pred_knc = knc_model.predict(X_test)

In [ ]:
f1_score_knc = f1_score(y_test, y_pred_knc)
print(f1_score_knc)

0.5952626158599381


In [ ]:
f1_scores["KNeighborsClassifier"] = f1_score_knc

# SVC

In [ ]:
# from sklearn.svm import SVC

# svc_grid = {'C': [0.1,1, 10, 100], 
#             'gamma': [1,0.1,0.01,0.001],
#             'kernel': ['rbf', 'poly', 'sigmoid']}

# svc_search = RandomizedSearchCV(estimator=SVC(),
#                           param_distributions=svc_grid,
#                           cv=5,
#                           scoring='f1',
#                           verbose=True)

# svc_search.fit(X_train, y_train)
# svc_best_params = svc_search.best_params_

In [ ]:
from sklearn.svm import SVC

svc_model = SVC()
svc_model.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
y_pred_svc = svc_model.predict(X_test)

In [ ]:
f1_score_svc = f1_score(y_test, y_pred_svc)
print(f1_score_svc)

0.7660311958405545


In [ ]:
f1_scores["SVC"] = f1_score_svc

# Decision Tree

In [ ]:
# from sklearn.tree import DecisionTreeClassifier

# dtc_grid = {"max_depth": [3, None],
#               "max_features": randint(1, 9),
#               "min_samples_leaf": randint(1, 9),
#               "criterion": ["gini", "entropy"]}

# dtc_search = RandomizedSearchCV(estimator=DecisionTreeClassifier(),
#                           param_distributions=dtc_grid,
#                           cv=5,
#                           scoring='f1',
#                           verbose=True)

# dtc_search.fit(X_train, y_train)
# dtc_best_params = dtc_search.best_params_

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtc_model = DecisionTreeClassifier()
dtc_model.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
y_pred_dtc = dtc_model.predict(X_test)

In [ ]:
f1_score_dtc = f1_score(y_test, y_pred_dtc)
print(f1_score_dtc)

0.7054202670856244


In [ ]:
f1_scores["DecisionTreeClassifier"] = f1_score_dtc

# Ensemble

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# rfc_grid = {"n_estimators": [10, 100, 200, 500, 1000, 1200],
#             "max_depth": [5, 8, 15, 25, 30],
#             "max_features": ["auto", "sqrt"],
#             "min_samples_split": [2, 5, 10, 15, 100],
#             "min_samples_leaf": [1, 2, 5, 10]}

# rfc_search = RandomizedSearchCV(estimator=RandomForestClassifier(),
#                           param_distributions=rfc_grid,
#                           cv=5,
#                           scoring='f1',
#                           verbose=True)

# rfc_search.fit(X_train, y_train)
# rfc_best_params = rfc_search.best_params_

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc_model = RandomForestClassifier()
rfc_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
y_pred_rfc = rfc_model.predict(X_test)

In [ ]:
f1_score_rfc = f1_score(y_test, y_pred_rfc)
print(f1_score_rfc)

0.7410423452768731


In [ ]:
f1_scores["RandomForestClassifier"] = f1_score_rfc

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb_model = GaussianNB()
gnb_model.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
y_pred_gnb = gnb_model.predict(X_test)

In [ ]:
f1_score_gnb = f1_score(y_test, y_pred_gnb)
print(f1_score_gnb)

0.7115044247787611


In [ ]:
f1_scores["GaussianNB"] = f1_score_gnb

In [ ]:
print(f1_scores)

{'LogisticRegression': 0.7360655737704918, 'KNeighborsClassifier': 0.5952626158599381, 'SVC': 0.7660311958405545, 'DecisionTreeClassifier': 0.7054202670856244, 'RandomForestClassifier': 0.7410423452768731, 'GaussianNB': 0.7115044247787611}


In [ ]:
len(X[0])

1721

# ANN

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall

def precision_m(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  return precision

def f1_m(y_true, y_pred):
  precision = precision_m(y_true, y_pred)
  recall = recall_m(y_true, y_pred)
  return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
import tensorflow as tf

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=1721, activation="relu")) # Rectifier linear unit
ann.add(tf.keras.layers.Dense(units=1721, activation="relu"))
ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1_m])

In [ ]:
ann.fit(X_train, y_train, batch_size=32, epochs=25)

Epoch 1/25
189/189 [==============================] - 1s 4ms/step - loss: 0.5005 - f1_m: 0.6849
Epoch 2/25
189/189 [==============================] - 1s 4ms/step - loss: 0.3053 - f1_m: 0.8319
Epoch 3/25
189/189 [==============================] - 1s 3ms/step - loss: 0.1691 - f1_m: 0.9142
Epoch 4/25
189/189 [==============================] - 1s 3ms/step - loss: 0.0938 - f1_m: 0.9536
Epoch 5/25
189/189 [==============================] - 1s 3ms/step - loss: 0.0618 - f1_m: 0.9698
Epoch 6/25
189/189 [==============================] - 1s 3ms/step - loss: 0.0526 - f1_m: 0.9738
Epoch 7/25
189/189 [==============================] - 1s 3ms/step - loss: 0.0428 - f1_m: 0.9754
Epoch 8/25
189/189 [==============================] - 1s 3ms/step - loss: 0.0398 - f1_m: 0.9766
Epoch 9/25
189/189 [==============================] - 1s 3ms/step - loss: 0.0384 - f1_m: 0.9774
Epoch 10/25
189/189 [==============================] - 1s 3ms/step - loss: 0.0406 - f1_m: 0.9793
Epoch 11/25
189/189 [==================

In [ ]:
y_pred = ann.predict(X_test)
ann.evaluate(X_test, y_test, verbose=1)

48/48 [==============================] - 0s 5ms/step - loss: 1.8597 - f1_m: 0.7291


[1.859741449356079, 0.7291105389595032]

# The best result using this data was shown by SVC model, which is F1 = 0.766